# Hybridisation Explorer: from **sp** to **sp³d²**

*General Chemistry & Cyberinfrastructure Skills Module*


## Learning Objective
By the end of this activity, you should be able to **describe** orbital hybridisation and **determine** whether a particular atom is *sp*, *sp²*, *sp³*, *sp³d*, or *sp³d²* hybridised.


## Prerequisites
- Python ≥ 3.8
- **RDKit** for cheminformatics
- *(Optional)* **py3Dmol** for 3‑D visualisation

> **Tip:** If you’re running this on Google Colab, execute the installation cell below **first**.

In [1]:
# Uncomment the next line the first time you run this notebook (or on Colab)
# !pip install rdkit-pypi py3Dmol -q

from rdkit import Chem
from rdkit.Chem import AllChem

# Helper function to convert RDKit HybridizationType to a nice string
def hybrid_str(atom):
    return str(atom.GetHybridization()).split('.')[-1].lower()

# (Optional) quick viewer
try:
    import py3Dmol
    def show_3d(mol):
        m2 = Chem.AddHs(mol)
        AllChem.EmbedMolecule(m2, randomSeed=0xf00d)
        mb = Chem.MolToMolBlock(m2)
        v = py3Dmol.view()
        v.addModel(mb, 'mol')
        v.setStyle({'stick': {}})
        v.zoomTo()
        return v.show()
except ModuleNotFoundError:
    def show_3d(mol):
        print('🛈 Install py3Dmol for a 3‑D view.')


## Quick Concept Check 🔍
Hybridisation is a *model* that helps rationalise molecular shape and bonding.

| Hybridisation | Steric number* | Example geometry |
|---------------|---------------|------------------|
| sp            | 2             | Linear |
| sp²           | 3             | Trigonal planar |
| sp³           | 4             | Tetrahedral |
| sp³d          | 5             | Trigonal bipyramidal |
| sp³d²         | 6             | Octahedral |

*Steric number = *σ*‑bonds + lone‑pairs on the atom.

## Worked Example 1 — Methane
We’ll let RDKit calculate the hybridisation of each atom in methane (*CH₄*).

In [2]:
methane = Chem.MolFromSmiles('C')
for atom in methane.GetAtoms():
    print(f'Atom {atom.GetIdx():>2} ({atom.GetSymbol():<2}) → {hybrid_str(atom)}')

# Visualise the 3‑D structure (optional)
show_3d(methane)

Atom  0 (C ) → sp3


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Worked Example 2 — Ethene vs Acetylene
Ethene (*C₂H₄*) has a C=C double bond, whereas acetylene (*C₂H₂*) has a C≡C triple bond.

In [3]:
for name, smiles in {'Ethene':'C=C', 'Acetylene':'C#C'}.items():
    mol = Chem.MolFromSmiles(smiles)
    hybs = {hybrid_str(a) for a in mol.GetAtoms() if a.GetSymbol() == 'C'}
    print(f'{name:10s}: carbon atoms → {", ".join(sorted(hybs))}')
    show_3d(mol)


Ethene    : carbon atoms → sp2


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Acetylene : carbon atoms → sp


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Worked Example 3 — Beyond Octet
Let’s inspect *PCl₅* (trigonal bipyramidal, sp³d) and *SF₆* (octahedral, sp³d²).

In [4]:
examples = {
    'Phosphorus pentachloride': 'ClP(Cl)(Cl)(Cl)Cl',
    'Sulfur hexafluoride'     : 'FS(F)(F)(F)(F)F'
}

for name, smi in examples.items():
    mol = Chem.MolFromSmiles(smi)
    # central atom is the first heavy atom RDKit parses
    central = mol.GetAtomWithIdx(1) if mol.GetAtomWithIdx(0).GetSymbol() == 'Cl' else mol.GetAtomWithIdx(0)
    print(f'{name}: central {central.GetSymbol()} → {hybrid_str(central)}')
    show_3d(mol)

Phosphorus pentachloride: central P → sp3d


[19:11:28] UFFTYPER: Warning: hybridization set to SP3 for atom 1


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Sulfur hexafluoride: central F → sp3


[19:11:28] UFFTYPER: Unrecognized atom type: S_6+6 (1)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Your Turn 📝
Pick **two** molecules you’re curious about (e.g. *CO₂*, *NH₃*, *XeF₂*…).
1. Create an RDKit molecule from SMILES.
2. Write a loop that prints the hybridisation of **every** atom.
3. Compare the results with VSEPR predictions.

*(💡 Need a hint? Revisit the `for`‑loop in the methane example.)*

In [5]:
# TODO 1: Replace with your own SMILES strings
my_smiles = ['CO2', 'N']  # ← EDIT ME

for smi in my_smiles:
    # TODO 2: Construct molecule and analyse hybridisation
    pass  # ← Your code here


### Challenge: Write a Helper Function
Write a function `predict_hybridisation(atom)` that returns the hybridisation based **only** on the steric number (σ bonds + lone pairs) *without* using RDKit’s built‑in method. Test it on a few molecules.

In [6]:
def predict_hybridisation(atom):
    """Return a string like 'sp3' based on steric number.*
    *Assumes atom has already computed valence & lone pair info.*"""
    # TODO 3: Implement logic here
    raise NotImplementedError


## Summary & Next Steps
- **Hybridisation** links atomic orbital mixing to molecular geometry.
- RDKit can programmatically report each atom’s hybridisation, enabling rapid checks across many structures.
- Challenge yourself to automate VSEPR‑based predictions **without** RDKit’s helper!

> **Extension:** Combine this workflow with data from databases (e.g. the *Materials Project* API) to analyse hybridisation trends across thousands of molecules.